# Recherche d'une fonction d'optimisation d'une architecture d'un réseau de neurones

## Analyse des differents types de DATASET

les 5 grands types de datasets couramment rencontrés dans le domaine de la science des données :

***Données Structurées*** :

- Ce sont des données qui ont une organisation fixe, généralement sous forme de tableaux avec des lignes et des colonnes. 
- Chaque colonne représente une variable et chaque ligne une observation.
Exemples : bases de données SQL, feuilles de calcul Excel, CSV.

***Données Non Structurées*** :

- Contrairement aux données structurées, les données non structurées ne suivent pas un modèle ou un format spécifique. 

- Elles sont plus complexes à analyser et à traiter.

Exemples : texte libre (emails, articles), images, vidéos, enregistrements audio.

***Données Semi-Structurées*** :

- Ces données ne sont pas organisées dans des tableaux rigides comme les données structurées, mais contiennent néanmoins des marqueurs ou des étiquettes qui séparent différents éléments.
Exemples : documents JSON, XML.

***Données Temporelles (Time Series)*** :

- Il s'agit de données qui sont collectées, enregistrées ou organisées en fonction du temps. 
- Elles sont utilisées pour analyser les tendances, les prévisions, etc.
Exemples : enregistrements boursiers, données météorologiques historiques, séries chronologiques de ventes.

***Données Spatiales (ou Géospatiales)*** :

- Ce type de données est lié à des informations géographiques ou spatiales. 
- Elles sont souvent utilisées pour des analyses cartographiques ou de localisation.
Exemples : cartes, données GPS, données de localisation de téléphones mobiles.


## Preprocessing top 5 par type de dataset

1. ***Données Structurées***
Nettoyage des Données :

- Gestion des valeurs manquantes : imputation, suppression des lignes/colonnes.
- Correction des erreurs et incohérences : valeurs aberrantes, erreurs de saisie.
- Transformation des Données :

- Normalisation/Standardisation : mise à l'échelle des variables pour une plage commune.
- Encodage : conversion des variables catégorielles en numériques (One-Hot Encoding, Label Encoding).
- Réduction de Dimensionnalité : Analyse en Composantes Principales (PCA), Sélection de Caractéristiques.

2. ***Données Non Structurées***
Texte :

- Nettoyage : suppression des balises HTML, des caractères spéciaux, et mise en minuscules.
- Tokenisation : découpage en mots ou phrases.
Suppression des mots vides (stop words), Stemming/Lemmatisation.
- Encodage : Bag of Words, TF-IDF, Word Embeddings.

*Images et Vidéos* :

- Normalisation : ajustement de la taille, mise à l'échelle des valeurs de pixels.
- Augmentation des données : rotation, translation, retournement pour augmenter la taille du dataset.
- Extraction de caractéristiques : utilisation de réseaux de neurones convolutifs (CNN).

*Audio* :

Extraction de caractéristiques : spectrogrammes, MFCC (Mel-Frequency Cepstral Coefficients).
Réduction du bruit.

3. ***Données Semi-Structurées***
- Extraction de Caractéristiques :
Conversion de formats JSON ou XML en structures tabulaires.
Extraction de champs spécifiques et transformation en données structurées pour une analyse plus poussée.

4. ***Données Temporelles (Time Series)***
Transformation :

Découpage en fenêtres temporelles.
Création de caractéristiques basées sur le temps : tendances, saisonnalité.
- Normalisation :

- Standardisation des séries temporelles.
- Décomposition :

Séparation des composantes tendancielles, saisonnières et résiduelles.

5. ***Données Spatiales (ou Géospatiales)***
Transformation des Coordonnées :

- Conversion des formats de coordonnées, ajustement aux systèmes de référence.
- Traitement des Données Raster et Vectorielles :

- Nettoyage, simplification des formes, superposition de couches d'informations.
- Extraction de Caractéristiques :

Création de caractéristiques basées sur la localisation (par exemple, distance à un point d'intérêt).

## Fonction d'évaluation du type de dataset

In [4]:
import pandas as pd
import numpy as np


def determine_dataset_type(dataset):
    # Vérifie si le dataset est un DataFrame pandas (données structurées)
    if isinstance(dataset, pd.DataFrame):
        # Vérifie la présence de colonnes temporelles
        if any(pd.api.types.is_datetime64_any_dtype(dataset[col]) for col in dataset.columns):
            return "Time Series (Temporal Data)"
        # Vérifie la présence de colonnes spatiales (latitude/longitude)
        if 'latitude' in dataset.columns and 'longitude' in dataset.columns:
            return "Spatial Data"
        return "Structured Data"

    # Vérifie si le dataset est une série de textes ou de fichiers (données non structurées)
    if isinstance(dataset, (list, pd.Series)) and all(isinstance(x, str) for x in dataset):
        return "Unstructured Data (Text or Files)"

    # Vérifie si le dataset est sous forme de dictionnaires ou de JSON (semi-structuré)
    if isinstance(dataset, (dict, list)) and all(isinstance(x, (dict, list)) for x in dataset):
        return "Semi-Structured Data (e.g., JSON)"

    return "Unknown Type"


# Exemple d'utilisation
# Remplacez par le chemin de votre fichier de données
dataset = pd.read_csv('SpotifyFeatures.csv')

print(determine_dataset_type(dataset))


Structured Data


1. Iris Dataset
Description : Contient des mesures de longueur et largeur de sépales et pétales pour trois espèces d'iris.
Type Attendu : Structured Data
2. Enron Email Dataset
Description : Collection d'environ 500 000 emails provenant de cadres de l'entreprise Enron.
Type Attendu : Unstructured Data (Text or Files)
3. New York City Taxi Trip Records
Description : Contient des enregistrements détaillés des trajets en taxi à New York, y compris les coordonnées de départ et d'arrivée.
Type Attendu : Spatial Data
4. Bitcoin Historical Data
Description : Données historiques sur le prix du Bitcoin, y compris la date et l'heure des transactions.
Type Attendu : Time Series (Temporal Data)
5. OpenStreetMap Data
Description : Contient des données géospatiales sous forme de fichiers XML, y compris des informations sur les routes, les bâtiments, etc.
Type Attendu : Semi-Structured Data (e.g., JSON) - Bien que les données OpenStreetMap soient souvent en XML, la fonction pourrait les identifier comme semi-structurées en raison de leur nature.

1. Iris Dataset


In [5]:

from sklearn import datasets
import pandas as pd

iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
print(determine_dataset_type(iris_df))


Structured Data


2. Enron Email Dataset (Simulation)

In [6]:
# Simuler un dataset de mails sous forme de liste de chaînes de caractères
# Ajoutez des exemples de mails
emails = ["Here is the meeting agenda...",
          "Please find attached the report...", ...]
print(determine_dataset_type(emails))


Unknown Type


3. New York City Taxi Trip Records (Simulation)

In [7]:
# Simuler un dataset de trajets de taxi avec des coordonnées
taxi_data = pd.DataFrame({
    'pickup_latitude': [40.761432, 40.644102, ...],  # Ajoutez des coordonnées
    'pickup_longitude': [-73.979815, -73.781632, ...],
    'dropoff_latitude': [40.641233, 40.729523, ...],
    'dropoff_longitude': [-73.958763, -73.991567, ...]
})
print(determine_dataset_type(taxi_data))


Structured Data


4. Bitcoin Historical Data (Simulation)

In [8]:
# Simuler un dataset de données historiques Bitcoin
btc_data = pd.DataFrame({
    'date': pd.date_range(start='2021-01-01', periods=100, freq='D'),
    'price': np.random.uniform(30000, 60000, 100)  # Prix aléatoires
})
print(determine_dataset_type(btc_data))


Time Series (Temporal Data)


5. OpenStreetMap Data (Simulation)

In [9]:
# Simuler un dataset de données géospatiales (XML/JSON-like structure)
osm_data = [
    {"type": "node", "id": 1, "lat": 59.941, "lon": 30.313},
    {"type": "node", "id": 2, "lat": 59.942, "lon": 30.314},
    ...  # Ajoutez des données fictives similaires
]
print(determine_dataset_type(osm_data))


Unknown Type


## Preprocessing des datasets
### Preprocessing pour les Données structurées

In [10]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


def preprocess_structured_data(dataset):
    # Prétraitement pour les données structurées
    imputer = SimpleImputer(strategy='mean')
    dataset = pd.DataFrame(imputer.fit_transform(
        dataset), columns=dataset.columns)
    scaler = StandardScaler()
    dataset = pd.DataFrame(scaler.fit_transform(
        dataset), columns=dataset.columns)
    return dataset


### Preprocessing pour les Données non structurées

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer


def preprocess_unstructured_data(dataset):
    # Prétraitement pour les données textuelles non structurées
    vectorizer = TfidfVectorizer(stop_words='english')
    dataset = vectorizer.fit_transform(dataset)
    return dataset


### Preprocessing pour les Données de type times series

In [12]:
def preprocess_temporal_data(dataset):
    # Prétraitement pour les données temporelles
    dataset.fillna(method='ffill', inplace=True)
    return dataset


### Preprocessing pour les Données Spatiales

In [13]:
from sklearn.preprocessing import MinMaxScaler


def preprocess_spatial_data(dataset):
    # Supposons que dataset est un DataFrame avec des colonnes 'latitude' et 'longitude'
    scaler = MinMaxScaler()
    dataset[['latitude', 'longitude']] = scaler.fit_transform(
        dataset[['latitude', 'longitude']])

    # Calcul d'une caractéristique dérivée, par exemple, la distance depuis le centre-ville
    city_center = (48.8566, 2.3522)  # Coordonnées de Paris, par exemple
    dataset['distance_from_center'] = np.sqrt(
        (dataset['latitude'] - city_center[0])**2 + (dataset['longitude'] - city_center[1])**2)

    return dataset


# Exemple d'utilisation
spatial_data = pd.DataFrame({
    'latitude': np.random.uniform(48.8, 48.9, 100),
    'longitude': np.random.uniform(2.3, 2.4, 100)
})
preprocessed_spatial_data = preprocess_spatial_data(spatial_data)


### preprocessing pour les Données Semi-Structurées (JSON)

In [14]:
import json
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


def preprocess_json_data(json_data):
    # Convertir le JSON en DataFrame
    dataset = pd.json_normalize(json_data)

    # Gérer les valeurs manquantes
    imputer = SimpleImputer(strategy='most_frequent')
    dataset = pd.DataFrame(imputer.fit_transform(
        dataset), columns=dataset.columns)

    # Encoder les variables catégorielles
    encoder = OneHotEncoder()
    encoded_columns = encoder.fit_transform(dataset[['interests']]).toarray()
    dataset = dataset.join(pd.DataFrame(
        encoded_columns, columns=encoder.get_feature_names_out(['interests'])))

    return dataset


# Exemple d'utilisation
json_data = [
    {"name": "Alice", "age": 30, "interests": "music"},
    {"name": "Bob", "age": 25, "interests": "sports"},
    # ... autres données
]
preprocessed_json_data = preprocess_json_data(json_data)


### Fonction globale de preprocessing

In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import datetime


def preprocess_dataset(dataset):
    dataset_type = determine_dataset_type(dataset)

    if dataset_type == "Structured Data":
        return preprocess_structured_data(dataset)

    elif dataset_type == "Unstructured Data (Text or Files)":
        return preprocess_unstructured_data(dataset)

    elif dataset_type == "Time Series (Temporal Data)":
        return preprocess_temporal_data(dataset)

    elif dataset_type == "Spatial Data":
        return preprocess_spatial_data(dataset)

    elif dataset_type == "Semi-Structured Data (e.g., JSON)":
        return preprocess_json_data(dataset)

    else:
        raise ValueError(
            "Type de dataset non reconnu ou non pris en charge pour le prétraitement")

    return dataset


In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import datetime


def preprocess_dataset(dataset):
    # Identifier le type de dataset
    dataset_type = determine_dataset_type(dataset)

    if dataset_type == "Structured Data":
        # Prétraitement pour les données structurées
        # Gérer les valeurs manquantes
        imputer = SimpleImputer(strategy='mean')
        dataset = pd.DataFrame(imputer.fit_transform(
            dataset), columns=dataset.columns)

        # Encoder les variables catégorielles
        dataset = pd.get_dummies(dataset)

        # Normaliser les données
        scaler = StandardScaler()
        dataset = pd.DataFrame(scaler.fit_transform(
            dataset), columns=dataset.columns)

    elif dataset_type == "Unstructured Data (Text or Files)":
        # Prétraitement pour les données textuelles non structurées
        vectorizer = TfidfVectorizer(stop_words='english')
        dataset = vectorizer.fit_transform(dataset)

    elif dataset_type == "Time Series (Temporal Data)":
        # Prétraitement pour les données temporelles
        # Remplir les valeurs manquantes
        dataset.fillna(method='ffill', inplace=True)

    elif dataset_type == "Spatial Data":
        # Prétraitement pour les données spatiales
        # Supposer que les données sont déjà sous forme numérique appropriée pour l'analyse
        pass

    elif dataset_type == "Semi-Structured Data (e.g., JSON)":
        # Prétraitement pour les données semi-structurées
        # Convertir en DataFrame (si nécessaire) et appliquer le prétraitement comme pour les données structurées
        # Cette étape dépendra fortement de la structure spécifique des données semi-structurées
        pass

    else:
        raise ValueError(
            "Type de dataset non reconnu ou non pris en charge pour le prétraitement")

    return dataset

# Utiliser la fonction sur un dataset spécifique
# dataset = ...  # Charger ou définir le dataset
# preprocessed_dataset = preprocess_dataset(dataset)


# Mise en place de l'API kaggle pour receuillir l'architecture des 300 meilleurs competiteurs

In [73]:
!pip install kaggle
!pip install selenium


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/97/e3/fd7272d6d2c49fd49a79a603cb28c8b5a71f8911861b4a0409b3c006a241/selenium-4.17.2-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/14/fb/9299cf74953f473a15accfdbe2c15218e766bae8c796f2567c83bae03e98/trio-0.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for typing_extensions>=4.9.0 from https://files.pythonhosted.org/packages/b7/f4/6a90020cd2d93349b442bfcb657d0dc91eee65491600b2cb1d388bc98e6b/typing_extensions-4.9.0-py3-none-any.whl.metada

In [77]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver

# Initialisation du navigateur sans spécifier le chemin d'accès du driver
browser = webdriver.Chrome()

# Votre code ici pour travailler avec le navigateur


# Chemin vers le driver de votre navigateur (par exemple, chromedriver pour Google Chrome)
# path_to_webdriver = 'chemin/vers/votre/chromedriver'

# Initialisation du service Chrome avec le chemin vers le chromedriver
service = Service(executable_path=path_to_webdriver)
browser = webdriver.Chrome(service=service)

# Accès à la page de classement Kaggle
browser.get('https://www.kaggle.com/rankings?group=competitions')

# Attendre que le contenu soit chargé
try:
    # Modifier la valeur de By.CLASS_NAME pour correspondre à la classe qui contient les utilisateurs
    element = WebDriverWait(browser, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'nom_de_classe_pour_les_utilisateurs'))
    )
    # Récupérer les informations des utilisateurs après le chargement du contenu
    user_elements = browser.find_elements(By.CLASS_NAME, 'nom_de_classe_pour_les_utilisateurs')
    for user_element in user_elements:
        # Extraire les informations ici
        pass
finally:
    browser.quit()

# Continuer le traitement des données récupérées


NoSuchDriverException: Message: Unable to locate or obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [71]:
user_list


[]

In [76]:
import requests
from bs4 import BeautifulSoup

# URL de la page de la compétition Kaggle
competition_url = "https://www.kaggle.com/c/how-to-preprocessing-when-using-embeddings/participants"  # Remplacez par l'URL de la compétition

# Envoyer une requête GET pour récupérer la page HTML
response = requests.get(competition_url)

# Vérifier si la requête a réussi
if response.status_code == 200:
    # Analyser la page HTML avec BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")

    # Trouver la section contenant la liste des compétiteurs
    competitors_section = soup.find("div", class_="ParticipantsHeader_title_1b6Gh")

    if competitors_section:
        # Extraire la liste des compétiteurs
        competitors_list = competitors_section.find_next("ul")

        # Afficher la liste des compétiteurs
        for competitor in competitors_list.find_all("li"):
            competitor_name = competitor.text.strip()
            print(f"Nom du compétiteur : {competitor_name}")
    else:
        print("La section des compétiteurs n'a pas été trouvée sur la page.")
else:
    print(f"Erreur lors de la requête GET : {response.status_code}")


Erreur lors de la requête GET : 404


In [57]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi
from tqdm import tqdm  # Importez tqdm

api = KaggleApi()
api.authenticate()

# # Obtenez la liste des compétiteurs de la compétition
# competitors = api.competition_list_teams(competition_name)

# for competitor in competitors:
#     print(competitor.userName, competitor.performanceTier)



# Lister les competitions disponibles
competitions = api.competitions_list()
for competition in tqdm(competitions):
    print(f"\ncompetition :\n{competition} \n")
    # Obtenez la liste des compétiteurs de la compétition
    competitors, _, _ = api.competition_list_teams_with_http_info(competition)
    # Affichez la liste des compétiteurs
    for competitor in competitors:
        print(f"Nom du compétiteur : {competitor['name']}")

# Lister les datasets disponibles
datasets = api.datasets_list()
# print(f"\n datasets:\n{datasets[0:10]} \n")
# info des 10 premiers
for dataset in tqdm(datasets[:10]):
    title = dataset.get('title', 'Titre non disponible')
    url = dataset.get('url', 'URL non disponible')
    print(f" Dataset Titre: {title}, URL: {url}")


for dataset in tqdm(datasets):
    title = dataset['titleNullable'] if 'titleNullable' in dataset else 'Titre non disponible'
    url = dataset['urlNullable'] if 'urlNullable' in dataset else 'URL non disponible'
    print(f" DAtaset Titre: {title}, URL: {url}")

kernels = api.kernels_list()
# print(f"\n kernels:\n{kernels[:10]} \n")
for kernel in kernel_objects:
    title = kernel.title
    author = kernel.author.get('userName', 'Auteur non disponible')
    url = kernel.url
    print(f"Titre: {title}, Auteur: {author}, URL: {url}")


  0%|          | 0/17 [00:00<?, ?it/s]


competition :
https://www.kaggle.com/competitions/blood-vessel-segmentation 



AttributeError: 'KaggleApi' object has no attribute 'competition_list_teams_with_http_info'

In [44]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()
# print(f"\napi :\n{api},api.authenticate(): {api.authenticate()} \n")

# Exemple pour récupérer une liste d'utilisateurs
# Note : Cette méthode ne garantit pas d'obtenir les 300 meilleurs compétiteurs
# users = api.users_list(page=1, max_results=300)
# for user in users:
#     print(user.userName, user.performanceTier)
# Lister les datasets disponibles
# Lister les datasets disponibles
datasets = api.datasets_list()
# print(f"\n datasets:\n{datasets} \n")
for dataset in datasets:
    title = dataset['titleNullable'] if 'titleNullable' in dataset else 'Titre non disponible'
    url = dataset['urlNullable'] if 'urlNullable' in dataset else 'URL non disponible'
    print(f"Titre: {title}, URL: {url}")



api :
<kaggle.api.kaggle_api_extended.KaggleApi object at 0x00000177BA716450>,api.authenticate(): None 

Titre: Jobs and Salaries in Data Science, URL: https://www.kaggle.com/datasets/hummaamqaasim/jobs-in-data
Titre: Apple Quality, URL: https://www.kaggle.com/datasets/nelgiriyewithana/apple-quality
Titre: 57,650 Spotify Songs, URL: https://www.kaggle.com/datasets/joebeachcapital/57651-spotify-songs
Titre: Gold Stock Prices, URL: https://www.kaggle.com/datasets/sahilwagh/gold-stock-prices
Titre: Heart Attack Prediction, URL: https://www.kaggle.com/datasets/juledz/heart-attack-prediction
Titre: TATA Motors Limited - Stock Prices (2006-2024), URL: https://www.kaggle.com/datasets/raunakpandey030/tata-motors-limited-stock-prices-2006-2024
Titre: New York Housing Market, URL: https://www.kaggle.com/datasets/nelgiriyewithana/new-york-housing-market
Titre: IMDB Movie Dataset Till Dec-2023, URL: https://www.kaggle.com/datasets/kianindeed/imdb-movie-dataset-dec-2023
Titre: NFL Team Data 2003-2

### Trie le top compétiteurs kaggle en fonction de leur médaille d'or et d'argent

In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Récupérer les variables d'environnement pour l'authentification
kaggle_username = os.getenv('KAGGLE_USERNAME')
kaggle_key = os.getenv('KAGGLE_KEY')

if not kaggle_username or not kaggle_key:
    raise Exception(
        "Les variables d'environnement KAGGLE_USERNAME et KAGGLE_KEY doivent être définies.")

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_key

api = KaggleApi()
api.authenticate()

# Récupérer une liste d'utilisateurs
users = api.users_list(page=1, max_results=300)

# Créer une liste pour stocker les informations des utilisateurs
user_medals = []

for user in users:
    # Récupérer le nombre de médailles d'or et d'argent
    gold_medals = user.competitionGoldMedals
    silver_medals = user.competitionSilverMedals

    # Ajouter les informations à la liste
    user_medals.append((user.userName, gold_medals, silver_medals))

# Trier les utilisateurs par le nombre de médailles d'or, puis par argent
sorted_users = sorted(user_medals, key=lambda x: (-x[1], -x[2]))

# Afficher les utilisateurs triés
for user in sorted_users[:300]:  # Limiter à 300 meilleurs
    print(f"User: {user[0]}, Gold Medals: {user[1]}, Silver Medals: {user[2]}")


## Les 5 grands type d'architecture de réseau de neurones

# Fonction création architecture réseau de neurones

### Version Optimisée

version avec moins de paramétres afin déavaluer le temps si l'on met tous les metriques
dans notre cas nous aurons 2*2*2*2*2*2*2 itérations= 128

In [1]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow import keras
import time
from tqdm import tqdm

# Charger l'ensemble de données Fashion MNIST
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
# Normaliser les données
X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Modifier les listes d'hyperparamètres pour réduire les itérations
taux_apprentissage = [0.01, 0.001]  # réduit à 2 options
batch_sizes = [64, 128]  # réduit à 2 options
architectures_couches = [[64], [128, 64]]  # réduit à 2 options
taux_dropout = [0.2, 0.3]  # réduit à 2 options
fonctions_perte = ['categorical_crossentropy',
                   'mean_squared_error']  # réduit à 2 options
optimiseurs = [keras.optimizers.Adam,
               keras.optimizers.SGD]  # réduit à 2 options
fonctions_activation = ['relu', 'sigmoid']  # réduit à 2 options

meilleurs_resultats = []
meilleure_accuracy = 0

start_time = time.perf_counter()

for lr in tqdm(taux_apprentissage):
    for batch_size in batch_sizes:
        for architecture in architectures_couches:
            for dropout in taux_dropout:
                for loss_function in fonctions_perte:
                    for Optimizer in optimiseurs:
                        for activation_function in fonctions_activation:
                            iteration_start_time = time.perf_counter()
                            # Votre code pour la construction, la compilation et l'entraînement du modèle...
                            # Construction et compilation du modèle
                            net = keras.models.Sequential()
                            net.add(keras.layers.Flatten(input_shape=(28, 28)))
                            for neurones in architecture:
                                net.add(keras.layers.Dense(
                                    neurones, activation=activation_function))
                                net.add(keras.layers.Dropout(dropout))
                            net.add(keras.layers.Dense(
                                10, activation='softmax'))
                            optimizer = Optimizer(learning_rate=lr)
                            net.compile(optimizer=optimizer,
                                        loss=loss_function, metrics=['accuracy'])

                            # Callback pour l'arrêt prématuré
                            early_stopping_cb = keras.callbacks.EarlyStopping(
                                patience=5, restore_best_weights=True)

                            # Entraînement du modèle
                            history = net.fit(X_train, y_train, epochs=nombre_epochs, batch_size=batch_size,
                                              validation_split=0.2, callbacks=[early_stopping_cb], verbose=0)

                            # Évaluer la performance du modèle
                            current_accuracy = max(
                                history.history['val_accuracy'])

                            # Comparer et stocker le meilleur résultat
                            if current_accuracy > meilleure_accuracy:
                                meilleure_accuracy = current_accuracy
                                meilleurs_resultats = [
                                    ('taux_apprentissage', lr),
                                    ('batch_size', batch_size),
                                    ('architecture', architecture),
                                    ('dropout', dropout),
                                    ('fonction_perte', loss_function),
                                    ('optimiseur', Optimizer.__name__),
                                    ('fonction_activation', activation_function),
                                    ('accuracy', meilleure_accuracy)
                                ]

                            iteration_end_time = time.perf_counter()
                            print(f"Temps pour cette itération: {
                                  iteration_end_time - iteration_start_time} secondes")

# Votre code pour afficher les meilleurs hyperparamètres...
# Afficher les meilleurs hyperparamètres après toutes les itérations
print("Meilleurs hyperparamètres : ", meilleurs_resultats)


end_time = time.perf_counter()
print(f"Temps total d'exécution: {end_time - start_time} secondes")

# temps total pour les 128 itération =2h avec 59s en moyenne par itération


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]


NameError: name 'nombre_epochs' is not defined

## Calcul de la complexité d'une fonction

Nombre total d'itérations: Calculez le nombre total de combinaisons d'hyperparamètres en multipliant le nombre d'options pour chaque hyperparamètre. Dans votre cas :

* taux_apprentissage: 3 options
* batch_sizes: 3 options
* architectures_couches: 4 options
* taux_dropout: 3 options
* fonctions_perte: 6 options (7 moins 1 puisque vous avez enlevé une option)
* optimiseurs: 7 options
* fonctions_activation: 4 options

Le nombre total d'itérations est donc 3 x 3 x 4 x 3 x 6 x 7 x 4 = 18 144.

In [13]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow import keras
import time

start_time = time.perf_counter()

# Charger l'ensemble de données Fashion MNIST
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
# Normaliser les données si nécessaire
X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Listes d'hyperparamètres
nombre_epochs = 50
taux_apprentissage = [0.01, 0.001, 0.0001]
batch_sizes = [64, 128, 256]
architectures_couches = [[64], [128], [64, 64], [128, 64]]
taux_dropout = [0.2, 0.3, 0.5]
#  suppression de sparse_categorical_crossentropy car incompatible avec oneEncoder, attend des catégories
fonctions_perte = ['categorical_crossentropy', 'binary_crossentropy', 'mean_squared_error',
                   'mean_absolute_error', 'kullback_leibler_divergence', 'poisson', 'cosine_similarity']
# fonctions_perte = ['sparse_categorical_crossentropy', 'categorical_crossentropy', 'binary_crossentropy', 'mean_squared_error', 'mean_absolute_error', 'kullback_leibler_divergence', 'poisson', 'cosine_similarity']
optimiseurs = [keras.optimizers.Adam, keras.optimizers.SGD, keras.optimizers.RMSprop,
               keras.optimizers.Adagrad, keras.optimizers.Adadelta, keras.optimizers.Nadam, keras.optimizers.Ftrl]
fonctions_activation = ['relu', 'sigmoid', 'tanh', 'softmax']

meilleurs_resultats = []
meilleure_accuracy = 0

for lr in taux_apprentissage:
    for batch_size in batch_sizes:
        for architecture in architectures_couches:
            for dropout in taux_dropout:
                for loss_function in fonctions_perte:
                    for Optimizer in optimiseurs:
                        for activation_function in fonctions_activation:
                            # Construction et compilation du modèle
                            net = keras.models.Sequential()
                            net.add(keras.layers.Flatten(input_shape=(28, 28)))
                            for neurones in architecture:
                                net.add(keras.layers.Dense(
                                    neurones, activation=activation_function))
                                net.add(keras.layers.Dropout(dropout))
                            net.add(keras.layers.Dense(
                                10, activation='softmax'))
                            optimizer = Optimizer(learning_rate=lr)
                            net.compile(optimizer=optimizer,
                                        loss=loss_function, metrics=['accuracy'])

                            # Callback pour l'arrêt prématuré
                            early_stopping_cb = keras.callbacks.EarlyStopping(
                                patience=5, restore_best_weights=True)

                            # Entraînement du modèle
                            history = net.fit(X_train, y_train, epochs=nombre_epochs, batch_size=batch_size,
                                              validation_split=0.2, callbacks=[early_stopping_cb], verbose=0)

                            # Évaluer la performance du modèle
                            current_accuracy = max(
                                history.history['val_accuracy'])

                            # Comparer et stocker le meilleur résultat
                            if current_accuracy > meilleure_accuracy:
                                meilleure_accuracy = current_accuracy
                                meilleurs_resultats = [
                                    ('taux_apprentissage', lr),
                                    ('batch_size', batch_size),
                                    ('architecture', architecture),
                                    ('dropout', dropout),
                                    ('fonction_perte', loss_function),
                                    ('optimiseur', Optimizer.__name__),
                                    ('fonction_activation', activation_function),
                                    ('accuracy', meilleure_accuracy)
                                ]
                             # , la compilation et l'entraînement du modèle...
                            iteration_end_time = time.perf_counter()
                            print(f"Temps pour cette itération: {
                                  iteration_end_time - iteration_start_time} secondes")

# Afficher les meilleurs hyperparamètres après toutes les itérations
print("Meilleurs hyperparamètres : ", meilleurs_resultats)

end_time = time.perf_counter()
duration = end_time - start_time
print(f"Temps d'exécution du script: {duration} secondes")

# # Exemple d'utilisation
# dataset = pd.read_csv('your_dataset.csv')
# architectures = [[64, 64], [128, 64], [128, 128, 64]]
# metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'auc']
# evaluate_neural_networks(dataset, 'target_column', architectures, metrics)

# Le modele ne donne rien aprées 630 min


Temps pour cette itération: 76.19445170000836 secondes
Temps pour cette itération: 114.79364879999775 secondes
Temps pour cette itération: 161.74815749999834 secondes
Temps pour cette itération: 188.8438976000034 secondes


: 

22,28	57,21459677	128	temps	2,034296774	heures
25,09		19000		301,9659274	
75,32					
72,74					
24,49					
45,29					
78,3					
74,6					
30,22					
24,71					
73,71					
71,93					
25,04					
31,52					
72,55					
73,78					
57,43					
53,96					
103,5					
102,5					
49,42					
46,75					
104,9					
104,5					
38,32					
51,48					
102,9					
103,2					
27,85					
61,87					
102,8					
102,2					
14,27					
24,79					
44,37					
44,88					
11,88					
40,14					
44,11					
44,86					
22,48					
17,77					
45,65					
45,66					
21,93					
34,33					
44,75					
45,34					
23,82					
20,99					
62,67					
62,55					
23,56					
33,87					
62,32					
61,13					
27,08					
49,81					
65,26					
64,93					
29,04					
40,2					
65,78					
64,57					
39,61					
57,12					
72,25					
75,28					
42,48					
81,32					
77,13					
73,74					
40,03					
73,1					
71,96					
71,72					
36,84					
69,58					
72,98					
72,11					
71,62					
98,54					
98,6					
101,3					
57,29					
82,53					
96,86					
97,62					
49,86					
88,38					
94,58					
95,99					
69,36					
104,5					
95,28					
97,45					
34,96					
45,75					
49,37					
43,97					
30,87					
49,22					
44,59					
44,04					
26,7					
50,08					
44,05					
43,9					
26,48					
49,75					
43,86					
43,96					
29,41					
58,22					
57,74					
58,28					
47,01					
60,2					
58,74					
58,78					
31,93					
72,56					
63,41					
59,83					
